In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
from analysis.lib.pq import pq_tools;reload(pq_tools)
from analysis.lib.fitting import fit, common
from analysis.lib.purification import purify_TPQI_from_Puri_Data as TPQI; reload(TPQI)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline


thetas = ['pi/4','pi/5','pi/6','pi/8']

In [ ]:
filtered_dts, filtered_dts_offset = [], []

for theta in thetas:
    print 'processing theta ', theta

    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    
    all_lt4,offsets,offsets_ch1 = TPQI.get_tstamps_and_offsets(folder_lt4, contains = 'Purify', verbose = False)
    
    file_dts,file_dts_offset, file_filtered_dts,file_filtered_dts_offset = TPQI.TPQI_analysis(folder_lt4,all_lt4,offsets,offsets_ch1, Verbose = False)

    print len(file_dts),len(file_dts_offset),len(file_filtered_dts),len(file_filtered_dts_offset)

    filtered_dts.extend(file_filtered_dts)
    filtered_dts_offset.extend(file_filtered_dts_offset)
    
filtered_dts = np.array(filtered_dts)
filtered_dts_offset = np.array(filtered_dts_offset)


In [ ]:
### plot result

lim = 50

hist_binsize = 4
bins = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,hist_binsize)


golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*89
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))

lw = 1
fontsize = 10
markersize = 4

ax = plt.subplot(111)
y,x=np.histogram(filtered_dts, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
ax.plot(bin_centres,y)

y,x=np.histogram(filtered_dts_offset, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
ax.plot(bin_centres,y)


ax.set_xlabel('Difference time (ns)')
ax.set_ylabel('Coincidences')
# ax.set_range
ax.set_xlim([-lim,lim])
ax.set_ylim([0,120])

###some plot cosmetics
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)

# ax.legend(borderaxespad=0.,fontsize =fontsize-1,frameon=False)

plt.tight_layout()

save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

plt.savefig(os.path.join(save_folder,'TPQI.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.1)

plt.savefig(os.path.join(save_folder,'TPQI.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.1)

plt.show()
plt.close('all')



In [ ]:
counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))
counts_in_next_pulse = np.sum((filtered_dts_offset<200) & (filtered_dts_offset>-200))

print 'coincidence with next pulse',counts_in_next_pulse 
print 'coincidences in same pulse',counts_in_same_pulse

tail_lt3 = 12.0 #9.1975 for SIL 3
tail_lt4 = 6.0

p1 = tail_lt3
p2 = tail_lt4

Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
# Visibility = 0.5
print 'assumed tail LT3/LT4',tail_lt3,tail_lt4
print 'distiungishable expected vis', Visibility
print 'measured Visibility', counts_in_same_pulse/float(counts_in_next_pulse)
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility
print 'Vis error', np.sqrt((counts_in_same_pulse**2)/float(counts_in_next_pulse**3)+ \
                                 (counts_in_same_pulse)/float(counts_in_next_pulse**2))/Visibility


In [ ]:
def sweep_tpqi_param(param_name,sweep_pts):
    vis = []
    vis_u = []
    counts_filt = []
    counts_filt_next = []
    
    initialVal = tpqi_dict[param_name]
    for p in sweep_pts:
        tpqi_dict[param_name] = p
        # get dts
        dts,dts_offset, filtered_dts,filtered_dts_offset = TPQI.TPQI_analysis(folder_lt4,all_lt4,offsets,offsets_ch1,
                                               tpqi_dict['start_ch0'],tpqi_dict['start_ch0'],
                                               tpqi_dict['window_length'],
                                              tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'],
                                               Verbose = False)
        ## calc visibility
        counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))
        counts_in_next_pulse = np.sum((filtered_dts_offset<200) & (filtered_dts_offset>-200))

        tail_lt3 = 7.2 #9.1975 for SIL 2 of pippin
        tail_lt4 = 5.1514

        p1 = tail_lt3
        p2 = tail_lt4

        Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
        if counts_in_same_pulse != 0 and counts_in_next_pulse != 0:
            vis.append(1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility)
            vis_u.append(np.sqrt((counts_in_same_pulse**2)/float(counts_in_next_pulse**3)+ \
                                 (counts_in_same_pulse)/float(counts_in_next_pulse**2))/Visibility)
            
#             print counts_in_same_pulse, counts_in_next_pulse,vis_u[-1]
        else:
            vis.append(0)
            vis_u.append(0)

        counts_filt.append(counts_in_same_pulse)
        counts_filt_next.append(counts_in_next_pulse)
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.errorbar(sweep_pts,vis,yerr=vis_u)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Visibility')
    plt.show()
    plt.close('all')
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.plot(sweep_pts,counts_filt,sweep_pts,counts_filt_next)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Counts')
    plt.show()
    plt.close('all')
    
    tpqi_dict[param_name] = initialVal
# Error propagation for N1/N2     Sqrt[N2^2 \[Sigma]1^2 + N1^2 \[Sigma]2^2]/N2^2

In [ ]:
##### sweep tpqi parameters
tpqi_dict = {
    'start_ch0' : 2774.5e3,
    'window_length' : 40e3,
    'pulse_sep' : 700e3,
    'number_of_pulses' : 1}

sweep_tpqi_param('start_ch0',np.linspace(2771e3,2781e3,10))
sweep_tpqi_param('window_length',np.linspace(10e3,60e3,10))

In [ ]:
def sweep_tpqi_days(steps):
    vis = []
    vis_u = []
    counts_filt = []
    counts_filt_next = []

    total_tstamps = len(all_lt4)
    file_ints = (np.linspace(0,total_tstamps,steps+1)).astype(int)

    for j in range(len(file_ints)-1):
        # get dts
        dts,dts_offset, filtered_dts,filtered_dts_offset = TPQI.TPQI_analysis(folder_lt4,all_lt4[file_ints[j]:file_ints[j+1]]
                                              ,offsets[file_ints[j]:file_ints[j+1]],offsets_ch1[file_ints[j]:file_ints[j+1]],tpqi_dict['start_ch0'],
                                              tpqi_dict['start_ch0'],
                                               tpqi_dict['window_length'],
                                              tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'],
                                               Verbose = False)
        ## calc visibility
        counts_in_same_pulse = np.sum((filtered_dts<200) & (filtered_dts>-200))
        counts_in_next_pulse = np.sum((filtered_dts_offset<200) & (filtered_dts_offset>-200))

        tail_lt3 = 7.2 #9.1975 for SIL 2 of pippin
        tail_lt4 = 5.1514

        p1 = tail_lt3
        p2 = tail_lt4

        Visibility = 2*p1*p2/(2*p1*p2 + p1**2 + p2**2)
        if counts_in_same_pulse != 0 and counts_in_next_pulse != 0:
            vis.append(1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility)
            vis_u.append(np.sqrt((counts_in_same_pulse**2)/float(counts_in_next_pulse**3)+ \
                                 (counts_in_same_pulse)/float(counts_in_next_pulse**2))/Visibility)
            
#             print counts_in_same_pulse, counts_in_next_pulse,vis_u[-1]
        else:
            vis.append(0)
            vis_u.append(0)

        counts_filt.append(counts_in_same_pulse)
        counts_filt_next.append(counts_in_next_pulse)
    
    sweep_pts = file_ints[1:]
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.errorbar(sweep_pts,vis,yerr=vis_u)
    ax.set_xlabel('File nos')
    ax.set_ylabel('Visibility')
    plt.show()
    plt.close('all')

    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot()
    plt.plot(sweep_pts,counts_filt,sweep_pts,counts_filt_next)
    ax.set_xlabel('File nos')
    ax.set_ylabel('Counts')
    plt.show()
    plt.close('all')
    

In [ ]:
##### sweep tpqi parameters
tpqi_dict = {
    'start_ch0' : 2774.5e3,
    'window_length' : 40e3,
    'pulse_sep' : 700e3,
    'number_of_pulses' : 1}

sweep_tpqi_days(3)

In [ ]:
### regular histgram plotting:
reload(pq_plots)
# f = tb.latest_data('105117',folder = folder_lt4)
# a = pqsequence.PQSequenceAnalysis(f)
f = tb.latest_data(all_lt4[-1],folder = folder_lt4)
b = pqsequence.PQSequenceAnalysis(f)

# pq_plots.plot_photon_hist(a.pqf,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log = False)
pq_plots.plot_photon_hist(b.pqf,hist_binsize = 1e2,start=2.760e6,length=0.040e6,log = False)
# a.plot_histogram(0,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)
# a.plot_histogram(1,hist_binsize = 1e2,start=2.430e6,length=0.01e6,log_plot = False)